In [1]:
import numpy as np
import math
from random import seed
from random import random
from numba import jit

In [3]:
T_start = 1000
T_end = 1
cooling_factor = 0.99
nMarkov = 1000
from functions import read_tsp_file
coords = read_tsp_file("Assignment3/TSP-Configurations/eil51.tsp")

from functions import comp_shortest_path
itr, min_cost, optimal_list_cities = comp_shortest_path(T_start, T_end, cooling_factor, nMarkov, coords)


In [4]:
print(min_cost)

423.77585422955303


In [5]:
print(optimal_list_cities)

[31  0 21  1 10 37  4 36 16  3 17 46 11 45 50 26  5 47 22 23 42  6 25  7
 30 27  2 35 34 19 28 20 15 49 33 29  8 48  9 38 32 44 14 43 41 39 18 40
 12 24 13]


In [6]:
print(itr)

688
